In [7]:
import pandas as pd
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config, AdamW
from torch.utils.data import DataLoader, Dataset

import warnings
warnings.filterwarnings('ignore')

In [8]:
# from google.colab import drive
# drive.mount('/content/drive')
# file_path = "/content/drive/MyDrive/THWS/RAW_recipes.csv"

file_path = "./data/RAW_recipes.csv"

In [9]:
recipes_df = pd.read_csv(file_path)
recipes_df

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8
...,...,...,...,...,...,...,...,...,...,...,...,...
231632,zydeco soup,486161,60,227978,2012-08-29,"['ham', '60-minutes-or-less', 'time-to-make', ...","[415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0]",7,"['heat oil in a 4-quart dutch oven', 'add cele...",this is a delicious soup that i originally fou...,"['celery', 'onion', 'green sweet pepper', 'gar...",22
231633,zydeco spice mix,493372,5,1500678,2013-01-09,"['15-minutes-or-less', 'time-to-make', 'course...","[14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0]",1,['mix all ingredients together thoroughly'],this spice mix will make your taste buds dance!,"['paprika', 'salt', 'garlic powder', 'onion po...",13
231634,zydeco ya ya deviled eggs,308080,40,37779,2008-06-07,"['60-minutes-or-less', 'time-to-make', 'course...","[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]",7,"['in a bowl , combine the mashed yolks and may...","deviled eggs, cajun-style","['hard-cooked eggs', 'mayonnaise', 'dijon must...",8
231635,cookies by design cookies on a stick,298512,29,506822,2008-04-15,"['30-minutes-or-less', 'time-to-make', 'course...","[188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]",9,['place melted butter in a large mixing bowl a...,"i've heard of the 'cookies by design' company,...","['butter', 'eagle brand condensed milk', 'ligh...",10


In [10]:
recipes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   name            231636 non-null  object
 1   id              231637 non-null  int64 
 2   minutes         231637 non-null  int64 
 3   contributor_id  231637 non-null  int64 
 4   submitted       231637 non-null  object
 5   tags            231637 non-null  object
 6   nutrition       231637 non-null  object
 7   n_steps         231637 non-null  int64 
 8   steps           231637 non-null  object
 9   description     226658 non-null  object
 10  ingredients     231637 non-null  object
 11  n_ingredients   231637 non-null  int64 
dtypes: int64(5), object(7)
memory usage: 21.2+ MB


In [11]:
recipes_df.duplicated().sum()

0

In [12]:
recipes_df.drop(columns=['id', 'n_steps', 'minutes', 'contributor_id',
                         'submitted', 'tags', 'nutrition', 'n_ingredients', 'description'], inplace=True)

recipes_df.rename(columns={'name': 'dish_name',
                  'steps': 'recipe'}, inplace=True)
recipes_df

,dish_name,recipe,ingredients
0,arriba baked winter squash mexican style,"['make a choice and proceed with recipe', 'dep...","['winter squash', 'mexican seasoning', 'mixed ..."
1,a bit different breakfast pizza,"['preheat oven to 425 degrees f', 'press dough...","['prepared pizza crust', 'sausage patty', 'egg..."
2,all in the kitchen chili,"['brown ground beef in large pot', 'add choppe...","['ground beef', 'yellow onions', 'diced tomato..."
3,alouette potatoes,['place potatoes in a large pot of lightly sal...,"['spreadable cheese with garlic and herbs', 'n..."
4,amish tomato ketchup for canning,['mix all ingredients& boil for 2 1 / 2 hours ...,"['tomato juice', 'apple cider vinegar', 'sugar..."
...,...,...,...
231632,zydeco soup,"['heat oil in a 4-quart dutch oven', 'add cele...","['celery', 'onion', 'green sweet pepper', 'gar..."
231633,zydeco spice mix,['mix all ingredients together thoroughly'],"['paprika', 'salt', 'garlic powder', 'onion po..."
231634,zydeco ya ya deviled eggs,"['in a bowl , combine the mashed yolks and may...","['hard-cooked eggs', 'mayonnaise', 'dijon must..."
231635,cookies by design cookies on a stick,['place melted butter in a large mixing bowl a...,"['butter', 'eagle brand condensed milk', 'ligh..."


In [13]:
recipes_df["recipe"] = recipes_df["recipe"].apply(
    eval).apply(lambda x: ', '.join(x))


recipes_df["ingredients"] = recipes_df["ingredients"].apply(
    eval).apply(lambda x: ', '.join(x))
recipes_df

,dish_name,recipe,ingredients
0,arriba baked winter squash mexican style,"make a choice and proceed with recipe, dependi...","winter squash, mexican seasoning, mixed spice,..."
1,a bit different breakfast pizza,"preheat oven to 425 degrees f, press dough int...","prepared pizza crust, sausage patty, eggs, mil..."
2,all in the kitchen chili,"brown ground beef in large pot, add chopped on...","ground beef, yellow onions, diced tomatoes, to..."
3,alouette potatoes,place potatoes in a large pot of lightly salte...,"spreadable cheese with garlic and herbs, new p..."
4,amish tomato ketchup for canning,"mix all ingredients& boil for 2 1 / 2 hours , ...","tomato juice, apple cider vinegar, sugar, salt..."
...,...,...,...
231632,zydeco soup,"heat oil in a 4-quart dutch oven, add celery ,...","celery, onion, green sweet pepper, garlic clov..."
231633,zydeco spice mix,mix all ingredients together thoroughly,"paprika, salt, garlic powder, onion powder, dr..."
231634,zydeco ya ya deviled eggs,"in a bowl , combine the mashed yolks and mayon...","hard-cooked eggs, mayonnaise, dijon mustard, s..."
231635,cookies by design cookies on a stick,place melted butter in a large mixing bowl and...,"butter, eagle brand condensed milk, light brow..."


In [14]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [15]:
class RecipeDataset(Dataset):
    def __init__(self, dish_name, recipe, ingredients, tokenizer, max_len):
        self.dish_name = dish_name
        self.recipe = recipe
        self.ingredients = ingredients
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.recipe)

    def __getitem__(self, idx):
        text = f"{self.dish_name[idx]} {self.recipe[idx]} {self.ingredients[idx]}"
        inputs = self.tokenizer.encode_plus(
            text, None, add_special_tokens=True, max_length=self.max_len, padding='max_length', truncation=True)

        ids = torch.tensor(inputs['input_ids'], dtype=torch.long)
        mask = torch.tensor(inputs['attention_mask'], dtype=torch.long)

        # Clip the input ids to ensure they are within the valid range
        ids = torch.clip(ids, 0, 50256)

        return {
            'ids': ids,
            'mask': mask
        }

In [16]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
max_len = 1024


dataset = RecipeDataset(recipes_df.dish_name.values, recipes_df.recipe.values,
                        recipes_df.ingredients.values, tokenizer, max_len)


dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [17]:
model = GPT2LMHeadModel.from_pretrained("gpt2", n_layer=3).to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)
criterion = torch.nn.CrossEntropyLoss()

Some weights of the model checkpoint at gpt2 were not used when initializing GPT2LMHeadModel: ['h.3.attn.c_proj.bias', 'h.10.ln_1.bias', 'h.9.attn.c_proj.bias', 'h.8.attn.c_attn.bias', 'h.4.attn.c_attn.weight', 'h.3.ln_1.weight', 'h.4.ln_1.bias', 'h.4.mlp.c_proj.weight', 'h.4.attn.bias', 'h.6.attn.c_proj.weight', 'h.3.attn.c_attn.weight', 'h.4.attn.c_attn.bias', 'h.5.ln_2.weight', 'h.8.ln_2.weight', 'h.4.mlp.c_fc.weight', 'h.8.attn.c_proj.weight', 'h.7.attn.c_proj.weight', 'h.8.mlp.c_fc.bias', 'h.4.mlp.c_proj.bias', 'h.7.attn.c_proj.bias', 'h.7.ln_1.weight', 'h.6.ln_1.weight', 'h.6.mlp.c_fc.weight', 'h.3.attn.c_proj.weight', 'h.9.attn.c_attn.weight', 'h.10.attn.c_proj.bias', 'h.7.mlp.c_fc.weight', 'h.5.mlp.c_fc.bias', 'h.10.attn.c_attn.bias', 'h.10.ln_2.bias', 'h.4.attn.c_proj.bias', 'h.11.mlp.c_fc.bias', 'h.9.attn.bias', 'h.9.ln_2.weight', 'h.7.attn.c_attn.weight', 'h.5.attn.bias', 'h.10.ln_2.weight', 'h.6.attn.c_attn.weight', 'h.3.attn.bias', 'h.11.ln_2.weight', 'h.7.mlp.c_proj.bias'

In [18]:
from tqdm import tqdm
epochs = 2

for epoch in range(epochs):
    model.train()
    loop = tqdm(dataloader)
    for batch in loop:
        ids = batch['ids'].to(device, dtype=torch.long)
        mask = batch['mask'].to(device, dtype=torch.long)

        outputs = model(ids, attention_mask=mask, labels=ids)
        loss = outputs[0]

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

Epoch 0:   0%|          | 14/115819 [02:26<316:52:13,  9.85s/it, loss=1.65] 

In [ ]:
# Save the fine-tuned model
model.save_pretrained("fine_tuned_gpt_model")
tokenizer.save_pretrained("fine_tuned_gpt_model")

In [ ]:
# User input and prediction
user_input_ingredients = input("Enter ingredients (comma-separated): ")
user_ingredients_list = [ingredient.strip()
                         for ingredient in user_input_ingredients.split(',')]

In [ ]:
# Load the fine-tuned model for prediction
fine_tuned_model = GPT2LMHeadModel.from_pretrained(
    "fine_tuned_model").to(device)
fine_tuned_tokenizer = GPT2Tokenizer.from_pretrained("fine_tuned_model")

In [ ]:
# Generate dish name and relevant recipe
user_input_text = ", ".join(user_ingredients_list)
input_ids = fine_tuned_tokenizer.encode(
    user_input_text, return_tensors="pt").to(device)
dish_name_output = fine_tuned_model.generate(
    input_ids, max_length=50, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95, early_stopping=True)

In [ ]:
dish_name = fine_tuned_tokenizer.decode(
    dish_name_output[0], skip_special_tokens=True)

In [ ]:
recipe_input_text = f"{dish_name} recipe {user_input_text}"
recipe_input_ids = fine_tuned_tokenizer.encode(
    recipe_input_text, return_tensors="pt").to(device)
recipe_output = fine_tuned_model.generate(
    recipe_input_ids, max_length=200, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95, early_stopping=True)

In [ ]:
recipe = fine_tuned_tokenizer.decode(
    recipe_output[0], skip_special_tokens=True)

In [ ]:
print("Generated Dish Name:", dish_name)
print("Generated Relevant Recipe:")
print(recipe)